# Module 12 Challenge
## Deliverable 2: Scrape and Analyze Mars Weather Data

In [ ]:
# Import relevant libraries
from splinter import Browser
from bs4 import BeautifulSoup as soup
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
browser = Browser('chrome')

### Step 1: Visit the Website

Use automated browsing to visit the [Mars Temperature Data Site](https://static.bc-edx.com/data/web/mars_facts/temperature.html). Inspect the page to identify which elements to scrape.

   > **Hint** To identify which elements to scrape, you might want to inspect the page by using Chrome DevTools to discover whether the table contains usable classes.


In [ ]:
# Visit the website
# https://static.bc-edx.com/data/web/mars_facts/temperature.html
url = "https://static.bc-edx.com/data/web/mars_facts/temperature.html"
browser.visit(url)

### Step 2: Scrape the Table

Create a Beautiful Soup object and use it to scrape the data in the HTML table.

Note that this can also be achieved by using the Pandas `read_html` function. However, use Beautiful Soup here to continue sharpening your web scraping skills.

In [ ]:
# Create a Beautiful Soup Object
html = browser.html
mars_soup = soup(html, 'html.parser')

In [ ]:
# Extract all rows of data
data_rows = mars_soup.find_all('tr', class_='data-row')
#print(data_rows)

### Step 3: Store the Data

Assemble the scraped data into a Pandas DataFrame. The columns should have the same headings as the table on the website. Here’s an explanation of the column headings:

* `id`: the identification number of a single transmission from the Curiosity rover
* `terrestrial_date`: the date on Earth
* `sol`: the number of elapsed sols (Martian days) since Curiosity landed on Mars
* `ls`: the solar longitude
* `month`: the Martian month
* `min_temp`: the minimum temperature, in Celsius, of a single Martian day (sol)
* `pressure`: The atmospheric pressure at Curiosity's location

In [ ]:
# Find all the rows in the table
rows = mars_soup.find_all('tr')

# Extract the column names from the first row
header_row = rows[0]
column_names = [th.text for th in header_row.find_all('th')]

# skipping the header row initialize rows list
data_rows = rows[1:]

# Create an empty list
data = []
# Loop through the scraped data to create a list of rows
for row in data_rows:
    # Extract data from the row
    data_values = [td.text for td in row.find_all('td')]

    # Create a dictionary representing a row of data
    data_row = dict(zip(column_names, data_values))

    # Append the data row to the list of rows
    data.append(data_row)
    #print(data_row)

In [ ]:
# Create a Pandas DataFrame by using the list of rows and a list of the column names
data_df = pd.DataFrame(data, columns=column_names)

In [ ]:
# Confirm DataFrame was created successfully
data_df.head()

### Step 4: Prepare Data for Analysis

Examine the data types that are currently associated with each column. If necessary, cast (or convert) the data to the appropriate `datetime`, `int`, or `float` data types.

  > **Hint** You can use the Pandas `astype` and `to_datetime` methods to accomplish this task.


In [ ]:
# Examine data type of each column
print(data_df.dtypes)

In [ ]:
# Change data types for data analysis
# Cast 'terrestrial_date' column to datetime
data_df['terrestrial_date'] = pd.to_datetime(data_df['terrestrial_date'])

# Cast 'sol', 'ls', 'month' columns to int
data_df['sol'] = data_df['sol'].astype(int)
data_df['ls'] = data_df['ls'].astype(int)
data_df['month'] = data_df['month'].astype(int)

# Cast 'min_temp' and 'pressure' columns to float
data_df['min_temp'] = data_df['min_temp'].astype(float)
data_df['pressure'] = data_df['pressure'].astype(float)

In [ ]:
# Confirm type changes were successful by examining data types again
print(data_df.dtypes)

### Step 5: Analyze the Data

Analyze your dataset by using Pandas functions to answer the following questions:

1. How many months exist on Mars?
2. How many Martian (and not Earth) days worth of data exist in the scraped dataset?
3. What are the coldest and the warmest months on Mars (at the location of Curiosity)? To answer this question:
    * Find the average the minimum daily temperature for all of the months.
    * Plot the results as a bar chart.
4. Which months have the lowest and the highest atmospheric pressure on Mars? To answer this question:
    * Find the average the daily atmospheric pressure of all the months.
    * Plot the results as a bar chart.
5. About how many terrestrial (Earth) days exist in a Martian year? To answer this question:
    * Consider how many days elapse on Earth in the time that Mars circles the Sun once.
    * Visually estimate the result by plotting the daily minimum temperature.


In [ ]:
# 1. How many months are there on Mars?
# Count the number of records per unique month
records_per_month = data_df['month'].value_counts()

# Create a DataFrame with the counts and the unique months as columns
df_counts = pd.DataFrame({'count': records_per_month})

# Sort the DataFrame by the index (month) in ascending order
df_counts_sorted = df_counts.sort_index()

print(df_counts_sorted)

In [ ]:
# 2. How many Martian days' worth of data are there?
# Count the number of unique Martian days
num_days = data_df['sol'].nunique()

# Calculate the difference between the maximum and minimum Martian days
day_difference = data_df['sol'].max() - data_df['sol'].min()

print("Number of Martian Days recorded:", num_days)
print("Time span (max - min):", day_difference)

In [ ]:
# 3. What is the average low temperature by month?
avg_min_temp = data_df.groupby('month')['min_temp'].mean()

print(avg_min_temp)

In [ ]:
avg_min_temp.plot(kind='bar', figsize=(10, 6))
plt.xlabel('Month')
plt.ylabel('Average Minimum Temperature (C)')
plt.title('Average Minimum Temperature per Month')
plt.xticks(rotation=0)
plt.show()

In [ ]:
# Identify the coldest and hottest months in Curiosity's location
avg_temp_sorted = avg_min_temp.sort_values()
# Create a bar chart with sorted data
avg_temp_sorted.plot(kind='bar', figsize=(10, 6))
plt.xlabel('Month')
plt.ylabel('Average Minimum Temperature (C)')
plt.title('Average Minimum Temperature per Month (Sorted)')
plt.xticks(rotation=0)
plt.show()

### The coldest month is month 3 and the warmest month is month 8 at Curiosity's location.

In [ ]:
# 4. Average pressure by Martian month

avg_pressure = data_df.groupby('month')['pressure'].mean()

print(avg_pressure)

In [ ]:
# Plot the average pressure by month
avg_pressure.plot(kind='bar', figsize=(10, 6))
plt.xlabel('Month')
plt.ylabel('Average Atmospheric Pressure')
plt.title('Average Atmospheric Pressure per Month')
plt.xticks(rotation=0)
plt.show()

In [ ]:
# Sort the series by values in ascending order
avg_pressure_sorted = avg_pressure.sort_values(ascending=True)

# Create a bar chart with sorted data
avg_pressure_sorted.plot(kind='bar', figsize=(10, 6))
plt.xlabel('Month')
plt.ylabel('Average Atmospheric Pressure')
plt.title('Average Atmospheric Pressure per Month (Sorted)')
plt.xticks(rotation=0)
plt.show()

In [ ]:
# 5. How many terrestrial (earth) days are there in a Martian year?
#import mplcursors

data_df['terrestrial_date'] = pd.to_datetime(data_df['terrestrial_date'])

data_df.plot('terrestrial_date', 'min_temp')

plt.xlabel('Earth Days')
plt.ylabel('Minimum Temperature (C)')
plt.title('Minimum Temperature by Day')
plt.xticks(rotation=0)

#cursor = mplcursors.cursor(data_df, hover=True)




plt.show()

In [ ]:
import mplcursors

data_df['terrestrial_date'] = pd.to_datetime(data_df['terrestrial_date'])
fig, ax = plt.subplots()
line = ax.plot(data_df['terrestrial_date'], data_df['min_temp'])

plt.xlabel('Earth Days')
plt.ylabel('Minimum Temperature (C)')
plt.title('Minimum Temperature by Day')
plt.xticks(rotation=0)

cursor = mplcursors.cursor(line, hover=True)

@cursor.connect("add")
def on_add(sel):
    date = data_df['terrestrial_date'][sel.target.index]
    temp = data_df['min_temp'][sel.target.index]
    sel.annotation.set_text(f'Date: {date}\nTemperature: {temp}°C')


plt.show()

### 5. How many terrestrial (earth) days are there in a Martian year? The max temps or min temps seem to be just under 2 years apart or roughly 700 days per year.


On average, the third month has the coldest minimum temperature on Mars, and the eighth month is the warmest. But it is always very cold there in human terms!



Atmospheric pressure is, on average, lowest in the sixth month and highest in the ninth.

The distance from peak to peak is roughly 1425-750, or 675 days. A year on Mars appears to be about 700 days from the plot. Internet search confirms that a Mars year is equivalent to 687 earth days.

### Step 6: Save the Data

Export the DataFrame to a CSV file.

In [ ]:
# Write the data to a CSV
data_df.to_csv('data_df_output.csv', index=False)


In [ ]:
browser.quit()